## Importar Librerias

In [7]:
from psycopg2 import connect, Error
import pandas as pd
import csv
from io import StringIO
import glob
import functools

## Conexion

In [8]:
def getConnection():
    try:
        connection = connect(host='localhost',database='amazon',user='postgres', password='1234', port='5432')   
    except(Exception, Error) as error:
        connection.rollback()
        print("Error: %s" % error)
    return connection

## Insert query

In [9]:
def insert_query(query,data):
    try:
        con = getConnection()
        if con and query != '':
            cursor = con.cursor()
            cursor.execute(query,data)
        con.commit()
    except(Exception, Error) as error:
        print("Error: %s" % error)

## Cargar product_category

In [10]:
files = glob.glob('*.tsv')
query = "insert into category(category_name) values (%s)"

for file in files:
    start = len("amazon_reviews_us_")
    end = file.index("_v1_00.tsv")
    name = file[start:end].replace("_"," ")  
    print(name)
    insert_query(query, (name,))

Apparel
Automotive
Gift Card


## Cargar a la base de datos

In [11]:
def to_database(dataframe,table):
    try:
        output = StringIO()
        dataframe.to_csv(output, sep='\t', index = False, header = False, quoting=csv.QUOTE_NONE,escapechar='\\')
        output.seek(0)
        connection = getConnection()
        cursor = connection.cursor()
        cursor.copy_from(output,table,null='')
        connection.commit()
        cursor.close()
    except(Exception, Error) as error:
        print("Error: %s" % error)



## Cargar Customer

In [12]:


df_customers_id = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3, usecols=[1]), glob.glob('*.tsv')))

df_customers_id = df_customers_id.drop_duplicates(subset="customer_id")


to_database(df_customers_id,"customer")


## Cargar Product

In [13]:
df_products = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3,usecols=[3,4,5,6]),glob.glob('*.tsv')))
df_products = df_products.drop_duplicates(subset="product_id")

to_database(df_products,"product")

## Cargar Time

In [14]:
df_time = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3 ,usecols=[14]),glob.glob('*.tsv')))
df_time = df_time.drop_duplicates(subset="review_date").dropna()

to_database(df_time,"time")

## Cargar Review

In [15]:
#Se leen todos los archivos que se encuentren en la carpeta terminados en .tsv
#LOS BOLEANOS EN LA REVIEW HAY QUE CAMBIARLOS


df_review = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3,usecols=[0,1,2,3,7,8,9,10,11,12,13,14]),glob.glob('*.tsv')))

to_database(df_review,"review")